In [0]:
%python

"""
Setup up some objects
"""

from pyspark.sql.functions import schema_of_json

_json = """{
   "store":{
      "fruit": [
        {"weight":8,"type":"apple"},
        {"weight":9,"type":"pear"}
      ],
      "basket":[
        [1,2,{"b":"y","a":"x"}],
        [3,4],
        [5,6]
      ],
      "book":[
        {
          "author":"Nigel Rees",
          "title":"Sayings of the Century",
          "category":"reference",
          "price":8.95
        },
        {
          "author":"Herman Melville",
          "title":"Moby Dick",
          "category":"fiction",
          "price":8.99,
          "isbn":"0-553-21311-3"
        },
        {
          "author":"J. R. R. Tolkien",
          "title":"The Lord of the Rings",
          "category":"fiction",
          "reader":[
            {"age":25,"name":"bob"},
            {"age":26,"name":"jack"}
          ],
          "price":22.99,
          "isbn":"0-395-19395-8"
        }
      ],
      "bicycle":{
        "price":19.95,
        "color":"red"
      }
    },
    "owner":"mark",
    "zip code":"94025"
 }"""

# get the schema of the json
_schema = spark.sql(f"select schema_of_json('{_json}') as schema").collect()[0][0]
print(_schema)

In [0]:
%python

"""
Create a nested json table
Code is taken from https://docs.databricks.com/optimizations/semi-structured.html#create-a-table-with-highly-nested-data
"""

display(spark.sql(f"""CREATE OR REPLACE TABLE store_data
    USING delta
    TBLPROPERTIES('delta.columnMapping.mode' = 'name',
                  'delta.minReaderVersion' = '2',
                  'delta.minWriterVersion' = '5')
    AS 
      SELECT from_json('{_json}', '{_schema}') as raw
 """))

In [0]:
%python

"""
Multiply data
"""

for i in range(10):
  spark.sql("INSERT INTO store_data SELECT * FROM store_data")

In [0]:
%sql

--
-- Visualize the nested json data
--

SELECT *
FROM store_data;

In [0]:
%sql

--
-- Calculate the sum of the average age for all readers
--

WITH _DATA AS (
  SELECT explode(raw.store.book.reader.age[2]) as age,
         books.category as category
  FROM store_data
  LATERAL VIEW explode(raw.store.book) as books
  )
  SELECT category,
         avg(age) as average_age,
         stddev(age) as sd_age
  FROM _DATA
  where lower(category) in ('reference', 'fiction')
  group by category;

In [0]:
%sql

--
-- Calculate the sum of the average age for all readers (no photon)
--

WITH _DATA AS (
  SELECT explode(raw.store.book.reader.age[2]) as age,
         books.category as category
  FROM store_data
  LATERAL VIEW explode(raw.store.book) as books
  )
  SELECT category,
         avg(age) as average_age,
         stddev(age) as sd_age
  FROM _DATA
  where lower(category) in ('reference', 'fiction')
  group by category;